In [11]:
from torch.utils.data import DataLoader, SequentialSampler
from dataclass_paired_vanilla import PairedVanilla

# Test-Dataset erstellen (Pfad und Dictionaries müssen übereinstimmen)
precision = "allele"
embed_base_dir = f"../../data_10x/embeddings/paired/{precision}"


# Lade Dictionaries (falls benötigt)
traV_dict = {}  # Lade die Dictionaries entsprechend
traJ_dict = {}
trbV_dict = {}
trbJ_dict = {}
mhc_dict = {}

test_file_path = "../../data_10x/splitted_datasets/allele/paired/test.tsv"
test_dataset = PairedVanilla(test_file_path, embed_base_dir, traV_dict, traJ_dict, trbV_dict, trbJ_dict, mhc_dict)

# Test-Dataloader erstellen
SEQ_MAX_LENGTH = 200  # Passe dies an dein Training an
BATCH_SIZE = 1  # Evaluierung oft Batchgröße 1

def pad_collate(self, batch):
        epitope_embeddings, tra_cdr3_embeddings, trb_cdr3_embeddings = [], [], []
        v_alpha, j_alpha, v_beta, j_beta = [], [], [], []
        epitope_sequence, tra_cdr3_sequence, trb_cdr3_sequence = [], [], []
        mhc = []
        task = []
        labels = []

        for item in batch:
            epitope_embeddings.append(item["epitope_embedding"])
            epitope_sequence.append(item["epitope_sequence"])
            tra_cdr3_embeddings.append(item["tra_cdr3_embedding"])
            tra_cdr3_sequence.append(item["tra_cdr3_sequence"])
            trb_cdr3_embeddings.append(item["trb_cdr3_embedding"])
            trb_cdr3_sequence.append(item["trb_cdr3_sequence"])
            v_alpha.append(item["v_alpha"])
            j_alpha.append(item["j_alpha"])
            v_beta.append(item["v_beta"])
            j_beta.append(item["j_beta"])
            mhc.append(item["mhc"])
            task.append(item["task"])
            labels.append(item["label"])

        max_length = self.seq_max_length

test_dataloader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    sampler=SequentialSampler(test_dataset),
    num_workers=0,  # Keep as 0 for compatibility
    collate_fn=pad_collate
)

#123

Error: File not found at path: ./data_10x/splitted_datasets/allele/paired/test.tsv
Keys in Epitope_paired_embeddings.npz: 1377
Keys in TRA_paired_embeddings.npz: 42011
Keys in TRB_paired_embeddings.npz: 45758
Unmapped TRA_CDR3 sequences: 0 []
Unmapped TRB_CDR3 sequences: 0 []
Unmapped Epitope sequences: 0 []


In [7]:
import os
print(os.path.abspath("./data_10x/splitted_datasets/allele/paired/test.tsv"))


/home/ubuntu/PA-Cancer-Immunotherapy-Transformer/BA_ZHAW/models/paired_vanilla/data_10x/splitted_datasets/allele/paired/test.tsv


In [13]:
import torch
from vanilla_model import VanillaModel  # Importiere die Modellstruktur

# Parameter wie beim Training definieren
EMBEDDING_SIZE = 1024
SEQ_MAX_LENGTH = 30  
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Lade die eingebetteten Dimensionen (z. B. falls diese Dictionaries vorhanden sind)
traV_embed_len = 26  # Beispielwert, setze diesen entsprechend dem Training
traJ_embed_len = 26
trbV_embed_len = 26
trbJ_embed_len = 26
mhc_embed_len = 26

# Setze die Hyperparameter entsprechend dem Training
hyperparameters = {
    "optimizer": "sgd",
    "learning_rate": 5e-3,
    "weight_decay": 0.075,
    "dropout_attention": 0.3,
    "dropout_linear": 0.45
}

# Erstelle eine Instanz des Modells
model = VanillaModel(
    EMBEDDING_SIZE, 
    SEQ_MAX_LENGTH, 
    DEVICE, 
    traV_embed_len, 
    traJ_embed_len, 
    trbV_embed_len, 
    trbJ_embed_len, 
    mhc_embed_len, 
    hyperparameters
)

# Lade die Modellgewichte
model.load_state_dict(torch.load("VanillaModel.pth", map_location=DEVICE), strict=True)

# Setze das Modell in den Evaluationsmodus
model.eval()

print("Modell erfolgreich geladen und im Evaluationsmodus.")


/tmp/ipykernel_37676/725910668.py:39: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("VanillaModel.pth", map_location=DEVICE), strict=True)


FileNotFoundError: [Errno 2] No such file or directory: 'VanillaModel.pth'

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

def evaluate_loaded_model(model, dataloader):
    true_labels = []
    predicted_labels = []

    with torch.no_grad():  # Keine Gradientenberechnung erforderlich
        for data in dataloader:
            inputs, labels = data  # Passe dies ggf. an deine Datenstruktur an
            outputs = model(inputs.to(DEVICE))
            _, predicted = torch.max(outputs, 1)

            true_labels.extend(labels.cpu().numpy())
            predicted_labels.extend(predicted.cpu().numpy())

    # Berechne Metriken
    conf_matrix = confusion_matrix(true_labels, predicted_labels)
    accuracy = accuracy_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels, zero_division=0)
    recall = recall_score(true_labels, predicted_labels, zero_division=0)

    print("Confusion Matrix:")
    print(conf_matrix)
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")

# Beispiel für den Aufruf der Evaluierungsfunktion
# Annahme: test_dataloader wurde bereits erstellt, wie in deinem Trainingsskript
evaluate_loaded_model(model, test_dataloader)
